# GEE Download

In [1]:
import ee
import geopandas as gpd

ee.Authenticate()
ee.Initialize(project="ee-tiagogoncalves")

In [2]:
from pathlib import Path

input_dir = Path("../data/raw")
output_dir = Path("../data/feature")

output_dir.mkdir(exist_ok=True)
(output_dir / 'bing').mkdir(exist_ok=True)
(output_dir / 'google').mkdir(exist_ok=True)

gpkg_files = list(input_dir.glob("*.gpkg"))

In [ ]:
def get_download_url(xmin, ymin, xmax, ymax, year):
    geometry = ee.Geometry.Rectangle([[xmin, ymin], [xmax, ymax]], proj='EPSG:3857')

    s_date = ee.Date.fromYMD(year, 1, 1)
    e_date = s_date.advance(1, 'year')
                
    collection = ee.ImageCollection("GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL")
    
    image = collection.filterBounds(geometry).filterDate(s_date, e_date).first()
    
    return image.getDownloadURL({
        'scale': 10,
        'region': geometry,
        'format': 'GEO_TIFF'
    })


def download_from_url(output_dir, url):
    import requests
    response = requests.get(url)
    
    if response.status_code == 200:
        with open(output_dir, 'wb') as f:
            f.write(response.content)
    else:
        raise Exception(f"status_code {response.status_code}")

In [8]:
import numpy as np

from tqdm import tqdm
from time import sleep

erros = []

for gpkg_file in tqdm(gpkg_files):
    gdf = gpd.read_file(gpkg_file)

    xmin, ymin, xmax, ymax = gdf.total_bounds

    for source in ['google', 'bing']:
        year = gdf.iloc[0][f'{source}_year']

        if np.isnan(year):
            continue
        else:
            year = int(year)

        if year < 2017 or year > 2024:
            continue

        for attempt in range(3):
            try:
                url = get_download_url(xmin, ymin, xmax, ymax, year)

                download_from_url(output_dir / source / f'{gpkg_file.stem}.tif', url)

                break
            except Exception as e:
                if attempt == 2:
                    erros.append((gpkg_file.stem, e))

                print(f"    ❌ Erro ao baixar a imagem: {e}. Tentando novamente...")
                sleep(2**attempt)

print("✅ Processamento concluído!")

  0%|          | 7/9799 [00:15<6:02:53,  2.22s/it]


KeyboardInterrupt: 